### Predicting If School is Public or Private

### Decision Tree (DT) vs Random Forest (RF) vs Gradient Boosted (GBT)

In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('private_vs_public').master('local[4]').getOrCreate()

22/06/14 10:44:02 WARN Utils: Your hostname, mehrdad-Standard resolves to a loopback address: 127.0.1.1; using 128.179.176.230 instead (on interface wlp4s0)
22/06/14 10:44:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/14 10:44:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/06/14 10:44:03 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/06/14 10:44:03 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/06/14 10:44:03 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/06/14 10:44:03 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
22/06/

### Loading Data

In [3]:
data = spark.read.csv('../data/College.csv', inferSchema=True, header=True)

In [4]:
data.count()

777

In [5]:
data.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)



In [6]:
data.describe().show()

+-------+--------------------+-------+------------------+------------------+----------------+------------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+----------------+------------------+
|summary|              School|Private|              Apps|            Accept|          Enroll|         Top10perc|         Top25perc|      F_Undergrad|      P_Undergrad|          Outstate|        Room_Board|             Books|          Personal|               PhD|          Terminal|         S_F_Ratio|       perc_alumni|          Expend|         Grad_Rate|
+-------+--------------------+-------+------------------+------------------+----------------+------------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+-------

In [7]:
data.columns

['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

### Assemble Features

In [8]:
from pyspark.ml.feature import VectorAssembler

In [9]:
inputCols = [
    'Apps',
    'Accept',
    'Enroll',
     'Top10perc',
     'Top25perc',
     'F_Undergrad',
     'P_Undergrad',
     'Outstate',
     'Room_Board',
     'Books',
     'Personal',
     'PhD',
     'Terminal',
     'S_F_Ratio',
     'perc_alumni',
     'Expend',
     'Grad_Rate']

In [10]:
assembler = VectorAssembler(inputCols=inputCols, outputCol='features')

In [11]:
output = assembler.transform(data)

In [12]:
output.columns

['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate',
 'features']

### Encode the Target Attribute

In [13]:
from pyspark.ml.feature import StringIndexer

In [14]:
indexer = StringIndexer(inputCol='Private', outputCol='PrivateIndex')

In [15]:
output_fixed = indexer.fit(output).transform(output)

In [16]:
output_fixed.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)
 |-- features: vector (nullable = true)
 |-- PrivateIndex: double (nullable = false)



In [17]:
final_data = output_fixed.select('features', 'PrivateIndex')

### Train Test Split

In [18]:
train_data, test_data = final_data.randomSplit([0.8, 0.2])

### Train Models

In [19]:
from pyspark.ml.classification import (
    DecisionTreeClassifier,
    RandomForestClassifier,
    GBTClassifier)

In [20]:
from pyspark.ml import Pipeline

In [21]:
dtc = DecisionTreeClassifier(labelCol='PrivateIndex', featuresCol='features')
rfc = RandomForestClassifier(labelCol='PrivateIndex', featuresCol='features')
gbt = GBTClassifier(labelCol='PrivateIndex', featuresCol='features')

In [22]:
dtc_model = dtc.fit(train_data)
rf_model = rfc.fit(train_data)
gbt_model = gbt.fit(train_data)

In [23]:
dtc_preds = dtc_model.transform(test_data)
rfc_preds = rf_model.transform(test_data)
gbt_preds = gbt_model.transform(test_data)

### Evaluation

In [24]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [25]:
bin_eval = BinaryClassificationEvaluator(labelCol='PrivateIndex')

In [26]:
print('DTC AUC', bin_eval.evaluate(dtc_preds))
print('RF AUC', bin_eval.evaluate(rfc_preds))
print('GBT AUC', bin_eval.evaluate(gbt_preds))

DTC AUC 0.9229508196721311
RF AUC 0.9761124121779862
GBT AUC 0.9632318501170961


In [27]:
# BUT WAIT, GBT Predictions don't have the Raw Predictions Column
gbt_preds.printSchema()

root
 |-- features: vector (nullable = true)
 |-- PrivateIndex: double (nullable = false)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [28]:
# This can cause errors when using BinaryClassificationEvaluator
# We need to define the name of rawPredictionCol to resolve this

In [29]:
bin_eval_2 = BinaryClassificationEvaluator(labelCol='PrivateIndex', rawPredictionCol='prediction')

In [30]:
print('GBT AUC', bin_eval_2.evaluate(gbt_preds))

GBT AUC 0.90807962529274


In [31]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [32]:
acc_eval = MulticlassClassificationEvaluator(labelCol='PrivateIndex',
                                            metricName='accuracy')

In [33]:
acc_eval.evaluate(gbt_preds)

0.9363057324840764